## Задание 2

Примените один из трансформеров, например BERT к задаче классификации отзывов клиентов. Сравните полученные результаты с классическими методами машинного обучения, с RNN. Сделайте выводы.

In [ ]:
!pip install transformers torch

In [ ]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification

import numpy as np
import re #регулярные выражения
from nltk.stem.snowball import RussianStemmer # стемминг
from nltk.stem import WordNetLemmatizer # лемматизация
from nltk.tokenize import word_tokenize # токенизация текста (разбиение на слова/токены)

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

from nltk.corpus import stopwords # стопслова для удаления из текста
from sklearn.model_selection import train_test_split # разделение выборки
# универсальная токенизация текста и токенизация под модели bert И gpt2
from transformers import AutoTokenizer, GPT2Tokenizer
# функция загрузки gpt-2 модели для генерации текста
from transformers import GPT2LMHeadModel, BertLMHeadModel
# функция добавления слоя в предобученную модель для классификации текста
from transformers import AutoModelForSequenceClassification
from transformers import pipeline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


### Загрузка датасета

In [ ]:
df = pd.read_csv("insurance_reviews.csv")
df

,header,bank,review,rate,date
0,Хорошая страховая компания,Т-Страхование,Оформили каско и не пожалели ! Когда случилос...,5.0,22.09.2024
1,За стразовку,Совкомбанк Страхование,"Самая дешёвая страховка из всех, в Совкомбанк....",4.0,22.09.2024
2,Как я пришел за ОСАГО в Т-Банк,Т-Страхование,Здравствуйте уважаемые СММщики Т-Банка. Пришел...,1.0,22.09.2024
3,ОСАГО+КАСКОGO,АльфаСтрахование,"Добрый вечер, один свой автомобиль застраховал...",5.0,22.09.2024
4,Обман сбербанк страхование семейный актив,Сбербанк страхование жизни,"Дорогие человеки,ни в коем случае не связывайт...",1.0,22.09.2024
...,...,...,...,...,...
5665,"Страховала по ОСАГО, все выполнили",Ингосстрах,"Случайно стукнула другой автомобиль, когда вые...",5.0,27.05.2024
5666,Получила полный ремонт,СОГАЗ,3 мая у меня произошёл страховой случай (я пос...,5.0,27.05.2024
5667,"Берет СОГАЗ деньги легко, а выплачивает не очень",СОГАЗ,"Страхование кредита в ВТБ, [censored] страхов...",2.0,27.05.2024
5668,Гарантированные накопления с защитой,Ренессанс Жизнь,Я решила вложить денежные средства в программу...,5.0,27.05.2024


### Обработка данных

In [ ]:
df.shape

(5670, 5)

In [ ]:
df.dtypes

,0
header,object
bank,object
review,object
rate,float64
date,object


In [ ]:
df.describe()

,rate
count,5669.000000
mean,3.627800
std,1.850283
min,1.000000
25%,1.000000
50%,5.000000
75%,5.000000
max,5.000000


In [ ]:
df.nunique()

,0
header,4337
bank,63
review,5667
rate,5
date,235


In [ ]:
df["review"].duplicated().sum()

2

In [ ]:
df.isna().sum()

,0
header,0
bank,0
review,1
rate,1
date,1


В результате изучения датасета и его наполнения выяснилось:

* размерность датасета - `(9561, 5)`;
* оценка указана в числовом формате `int64`, остальные столбцы - `object`;
* средняя оценка по отзывам - `3,58`;
* из 9561 отзывов уникальными являются 9551, 10 отзывов - дубликаты;
* нулевые значения отсутствуют;
* количество банков/организаций в датасете - `65`.

Следовательно, дубликаты необходимо удалить. Далее будет обработка и очистка данных.

In [ ]:
# удаляем дубликаты отзывов
df.drop_duplicates(subset="review", inplace=True)
df

,header,bank,review,rate,date
0,Хорошая страховая компания,Т-Страхование,Оформили каско и не пожалели ! Когда случилос...,5.0,22.09.2024
1,За стразовку,Совкомбанк Страхование,"Самая дешёвая страховка из всех, в Совкомбанк....",4.0,22.09.2024
2,Как я пришел за ОСАГО в Т-Банк,Т-Страхование,Здравствуйте уважаемые СММщики Т-Банка. Пришел...,1.0,22.09.2024
3,ОСАГО+КАСКОGO,АльфаСтрахование,"Добрый вечер, один свой автомобиль застраховал...",5.0,22.09.2024
4,Обман сбербанк страхование семейный актив,Сбербанк страхование жизни,"Дорогие человеки,ни в коем случае не связывайт...",1.0,22.09.2024
...,...,...,...,...,...
5665,"Страховала по ОСАГО, все выполнили",Ингосстрах,"Случайно стукнула другой автомобиль, когда вые...",5.0,27.05.2024
5666,Получила полный ремонт,СОГАЗ,3 мая у меня произошёл страховой случай (я пос...,5.0,27.05.2024
5667,"Берет СОГАЗ деньги легко, а выплачивает не очень",СОГАЗ,"Страхование кредита в ВТБ, [censored] страхов...",2.0,27.05.2024
5668,Гарантированные накопления с защитой,Ренессанс Жизнь,Я решила вложить денежные средства в программу...,5.0,27.05.2024


In [ ]:
# изменение формата столбца даты с обходом и заменой строк, в которых дата указана некорректно
# (некорректные данные будут заменены на значение NaT)
df["date"] = pd.to_datetime(df["date"], format="%d.%m.%Y", errors='coerce')
df

,header,bank,review,rate,date
0,Хорошая страховая компания,Т-Страхование,Оформили каско и не пожалели ! Когда случилос...,5.0,2024-09-22
1,За стразовку,Совкомбанк Страхование,"Самая дешёвая страховка из всех, в Совкомбанк....",4.0,2024-09-22
2,Как я пришел за ОСАГО в Т-Банк,Т-Страхование,Здравствуйте уважаемые СММщики Т-Банка. Пришел...,1.0,2024-09-22
3,ОСАГО+КАСКОGO,АльфаСтрахование,"Добрый вечер, один свой автомобиль застраховал...",5.0,2024-09-22
4,Обман сбербанк страхование семейный актив,Сбербанк страхование жизни,"Дорогие человеки,ни в коем случае не связывайт...",1.0,2024-09-22
...,...,...,...,...,...
5665,"Страховала по ОСАГО, все выполнили",Ингосстрах,"Случайно стукнула другой автомобиль, когда вые...",5.0,2024-05-27
5666,Получила полный ремонт,СОГАЗ,3 мая у меня произошёл страховой случай (я пос...,5.0,2024-05-27
5667,"Берет СОГАЗ деньги легко, а выплачивает не очень",СОГАЗ,"Страхование кредита в ВТБ, [censored] страхов...",2.0,2024-05-27
5668,Гарантированные накопления с защитой,Ренессанс Жизнь,Я решила вложить денежные средства в программу...,5.0,2024-05-27


In [ ]:
# количество строк с некорректной датой
df["date"].isna().sum()

855

In [ ]:
df.dropna(axis=0, subset="date", inplace=True)
df

,header,bank,review,rate,date
0,Хорошая страховая компания,Т-Страхование,Оформили каско и не пожалели ! Когда случилос...,5.0,2024-09-22
1,За стразовку,Совкомбанк Страхование,"Самая дешёвая страховка из всех, в Совкомбанк....",4.0,2024-09-22
2,Как я пришел за ОСАГО в Т-Банк,Т-Страхование,Здравствуйте уважаемые СММщики Т-Банка. Пришел...,1.0,2024-09-22
3,ОСАГО+КАСКОGO,АльфаСтрахование,"Добрый вечер, один свой автомобиль застраховал...",5.0,2024-09-22
4,Обман сбербанк страхование семейный актив,Сбербанк страхование жизни,"Дорогие человеки,ни в коем случае не связывайт...",1.0,2024-09-22
...,...,...,...,...,...
5664,"Личный кабинет не работает, техподдержка отсут...",Росгосстрах,Личный кабинет не работает. При попытке зайти ...,1.0,2024-05-27
5665,"Страховала по ОСАГО, все выполнили",Ингосстрах,"Случайно стукнула другой автомобиль, когда вые...",5.0,2024-05-27
5666,Получила полный ремонт,СОГАЗ,3 мая у меня произошёл страховой случай (я пос...,5.0,2024-05-27
5667,"Берет СОГАЗ деньги легко, а выплачивает не очень",СОГАЗ,"Страхование кредита в ВТБ, [censored] страхов...",2.0,2024-05-27


Проверим количество отзывов для каждой организации и удалим те, у которых количество отзывов меньше 100.

In [ ]:
df_bank = df.groupby("bank").review.count().sort_values()
df_bank

,review
bank,
Ак Барс Страхование,1
Бестиншур,1
ДЕЛО ЖИЗНИ (НСГ Страхование жизни),1
НКО ПОВС «Страховой дом «Платинум»,1
Капитал МС (РГС-Медицина),1
...,...
АльфаСтрахование-Жизнь,221
СОГАЗ,276
Совкомбанк Страхование,611


In [ ]:
organizations_less_100_reviews = df_bank.loc[df_bank.index.isin(df_bank.where(df_bank < 100).dropna().index)].index
organizations_less_100_reviews

Index(['Ак Барс Страхование', 'Бестиншур',
       'ДЕЛО ЖИЗНИ (НСГ Страхование жизни)',
       'НКО ПОВС «Страховой дом «Платинум»', 'Капитал МС (РГС-Медицина)',
       'Ингосстрах-Жизнь', 'Зетта Страхование жизни',
       'Зетта Страхование (бывшая СК «Альянс»)', 'ППФ Страхование жизни',
       'Медицинская акционерная страховая компания (МАКС-М)',
       'РСХБ-Страхование жизни', 'РЕСО-Мед', 'Спасские ворота',
       'Страховая фирма «Адонис»', 'Инсайт', 'АльфаСтрахование-ОМС',
       'Русский Стандарт Страхование', 'Пари', 'Хоум Кредит Страхование',
       'Русское страховое общество «Евроинс»', 'Газпром Страхование',
       'Двадцать первый век', 'БКС Страхование жизни', 'Энергогарант',
       'ПСБ Страхование', 'Инлайф страхование жизни', 'СОГАЗ-Мед',
       'Капитал-Полис', 'Страховая бизнес группа', 'РСХБ-Страхование',
       'Объединенная страховая компания', 'ВСК-Линия жизни', 'МАКС-Жизнь',
       'Согласие-Вита', 'СК ГАЙДЕ', 'Д2 Страхование', 'Гелиос', 'Астро-Волга',
       '

In [ ]:
df = df[df.bank.isin(list(organizations_less_100_reviews)) == False]
df

,header,bank,review,rate,date
0,Хорошая страховая компания,Т-Страхование,Оформили каско и не пожалели ! Когда случилос...,5.0,2024-09-22
1,За стразовку,Совкомбанк Страхование,"Самая дешёвая страховка из всех, в Совкомбанк....",4.0,2024-09-22
2,Как я пришел за ОСАГО в Т-Банк,Т-Страхование,Здравствуйте уважаемые СММщики Т-Банка. Пришел...,1.0,2024-09-22
3,ОСАГО+КАСКОGO,АльфаСтрахование,"Добрый вечер, один свой автомобиль застраховал...",5.0,2024-09-22
4,Обман сбербанк страхование семейный актив,Сбербанк страхование жизни,"Дорогие человеки,ни в коем случае не связывайт...",1.0,2024-09-22
...,...,...,...,...,...
5664,"Личный кабинет не работает, техподдержка отсут...",Росгосстрах,Личный кабинет не работает. При попытке зайти ...,1.0,2024-05-27
5665,"Страховала по ОСАГО, все выполнили",Ингосстрах,"Случайно стукнула другой автомобиль, когда вые...",5.0,2024-05-27
5666,Получила полный ремонт,СОГАЗ,3 мая у меня произошёл страховой случай (я пос...,5.0,2024-05-27
5667,"Берет СОГАЗ деньги легко, а выплачивает не очень",СОГАЗ,"Страхование кредита в ВТБ, [censored] страхов...",2.0,2024-05-27


In [ ]:
# выведем среднее количество отзывов для организаций
mean_number_of_reviews = round(df.groupby("bank").review.count().mean(), 2)
mean_number_of_reviews

332.54

Выведем
* размерность датасета, очищенного от дубликатов, пустых значений, некорректной даты и организаций с малым количеством отзывов;
* количество уникальных значений для очищенного датасета;
* описательную характеристику нового датасета.


In [ ]:
df.shape

(4323, 5)

In [ ]:
df.describe()

,rate,date
count,4323.000000,4323
mean,3.756419,2024-08-03 09:14:16.904927232
min,1.000000,2024-05-27 00:00:00
25%,1.000000,2024-07-04 00:00:00
50%,5.000000,2024-08-07 00:00:00
75%,5.000000,2024-09-04 00:00:00
max,5.000000,2024-09-22 00:00:00
std,1.804829,NaN


In [ ]:
mean_value = round(df.describe()["rate"]["mean"], 2)
mean_value

3.76

В результате обработки данных:
* размерность датасета стала `(7430, 5)` - удалили `2131` строки;
* средняя оценка по выборке стала `3.73` - выросла на `0,15`;
* были удалены дубликаты отзывов;
* были удалены строки, в котрых дата была указана неправильно;
* были удалены организации, у которых количество отзывов было меньше 100.

Сначала напишем 2 функции, которые приведут все данные в нижний регистр, удалит стоп-слова и лишние символы, а так же проведет стемминг или лемматизацию.

### Нормализация слов

In [ ]:
df_stemming = df.copy()
df_lematization = df.copy()

In [ ]:
def prepare_data_stem(text, window_size=2):
  '''Функция обработки текстовых полей для более удобной работы далее'''
  # преобразуем текст в нижний регистр
  text = text.lower()

  # удаляем все символы кроме букв и пробелов
  text = re.sub(r'[^а-я\s]', ' ', text)

  # удаляем стопслова
  tokens = word_tokenize(text) # делим строку на токены
  filtered_tokens = [word for word in tokens if not word in stopwords.words('russian')]

  # стемминг
  st = RussianStemmer()
  stemmed_tockens = [st.stem(word) for word in filtered_tokens]
  result = " ".join(stemmed_tockens)
  return result

In [ ]:
df_stemming["review"] = df_stemming["review"].apply(prepare_data_stem)

In [ ]:
df_stemming

,header,bank,review,rate,date
0,Хорошая страховая компания,Т-Страхование,оформ каск пожалел случ дтп менеджер анастас с...,5,2024-09-22
1,За стразовку,Совкомбанк Страхование,сам деш ва страховк совкомбанк позвон сам помо...,4,2024-09-22
2,Как я пришел за ОСАГО в Т-Банк,Т-Страхование,здравств уважа сммщик т банк пришел знач сво л...,1,2024-09-22
3,ОСАГО+КАСКОGO,АльфаСтрахование,добр вечер сво автомобил застрахова альфастрах...,5,2024-09-22
4,Обман сбербанк страхование семейный актив,Сбербанк страхование жизни,дорог человек ко случа связыва сбербанк люб ст...,1,2024-09-22
...,...,...,...,...,...
9554,Тинькофф страхование,Т-Страхование,прошл быстр деньг поступ карт втор ден подач д...,5,2024-01-21
9556,Мне предоставили хорошую скидку на страховку ж...,Т-Страхование,позвон тинькофф страхован оформл ипотек днях д...,5,2024-01-20
9557,Мой выбор - Ренессанс,Ренессанс,здравств принят оставля положительн отзыв стра...,5,2024-01-20
9558,"Защита в пути, КАСКО",Сбербанк страхование,приобр л автомобил кред драйв клик банк услов ...,1,2024-01-20


In [ ]:
def prepare_data_lem(text, window_size=2):
  '''Функция обработки текстовых полей для более удобной работы далее'''
  # преобразуем текст в нижний регистр
  text = text.lower()

  # удаляем все символы кроме букв и пробелов
  text = re.sub(r'[^а-я\s]', ' ', text)

  # удаляем стопслова
  tokens = word_tokenize(text) # делим строку на токены
  filtered_tokens = [word for word in tokens if not word in stopwords.words('russian')]

  # лемматизация
  wnl = WordNetLemmatizer()
  lemmed_tockens = [wnl.lemmatize(word) for word in filtered_tokens]
  result = " ".join(lemmed_tockens)
  return result

In [ ]:
df_lematization["review"] = df_lematization["review"].apply(prepare_data_lem)

In [ ]:
df_lematization

,header,bank,review,rate,date
0,Хорошая страховая компания,Т-Страхование,оформили каско пожалели случилось дтп менеджер...,5,2024-09-22
1,За стразовку,Совкомбанк Страхование,самая деш вая страховка совкомбанк позвонили с...,4,2024-09-22
2,Как я пришел за ОСАГО в Т-Банк,Т-Страхование,здравствуйте уважаемые сммщики т банка пришел ...,1,2024-09-22
3,ОСАГО+КАСКОGO,АльфаСтрахование,добрый вечер свой автомобиль застраховал альфа...,5,2024-09-22
4,Обман сбербанк страхование семейный актив,Сбербанк страхование жизни,дорогие человеки коем случае связывайтесь сбер...,1,2024-09-22
...,...,...,...,...,...
9554,Тинькофф страхование,Т-Страхование,прошло быстро деньги поступили карту второй де...,5,2024-01-21
9556,Мне предоставили хорошую скидку на страховку ж...,Т-Страхование,позвонили тинькофф страхования оформлена ипоте...,5,2024-01-20
9557,Мой выбор - Ренессанс,Ренессанс,здравствуйте принято оставлять положительные о...,5,2024-01-20
9558,"Защита в пути, КАСКО",Сбербанк страхование,приобр л автомобиль кредит драйв клик банке ус...,1,2024-01-20


### SBert на необработанных данных

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
model = AutoModelForSequenceClassification.from_pretrained("ai-forever/sbert_large_nlu_ru", num_labels=5)  # Укажите количество классов

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/863 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/sbert_large_nlu_ru and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Пример текста отзыва
test_review_1 = df["review"][4] # 1
test_review_2 = df["review"][0] # 5
test_review_3 = df["review"][30] # 2

test_reviews = [test_review_1, test_review_2,test_review_3]

In [ ]:
for test in test_reviews:
  inputs = tokenizer(test, return_tensors='pt', padding=True, truncation=True)
  with torch.no_grad():
      outputs = model(**inputs)
      logits = outputs.logits

  # Получение предсказанного класса
  predictions = torch.argmax(logits, dim=-1)
  print(test)
  print(f"Оценка модели: {predictions[0] + 1}| " + "Оценка отзыва: ", df[df["review"] == test]["rate"].values[0])  # Добавляем 1, чтобы получить класс от 1 до 5


Дорогие человеки,ни в коем случае не связывайтесь со сбербанком по любому страхованию. В 2021 году менеджер сбербанка уговорила меня на накопительной страхование жизни. В глаза врала,что это этот вид страхования очень выгодный,мало того что вы, говорила она мне, будете застрахованы, так ещё и инвестиционный доход и налоговый вычет получите.и в любое время можете расторгнуть договор, получив все сумму .выгода банка это то что мои деньги будут крутиться.в итоге после 3 лет внесения взносов, я все таки решила узнать сколько я получу после 5 лет внесения взносов по договору. Позвонив по 900 ,я услышала что из 200000 внесённых мной взносов мне вернут только 100000 и это не при расторжении а по окончании действия договора.я была в шоке и тут же расторгла договор.в итоге из 145000 мной внесённых средств ,мне вернули только ,внимание!!!! 29000, это просто открытый грабёж сбербанка.люди ,не связывайтесь со сбербанком по страхованию.берегите свои нервы и деньги. Даже при посещение офис мне в гла

### SBert на обработанных данных (stemming)

In [ ]:
tokenizer_stem = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
model_stem = AutoModelForSequenceClassification.from_pretrained("ai-forever/sbert_large_nlu_ru", num_labels=5)  # Укажите количество классов

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/sbert_large_nlu_ru and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Пример текста отзыва
test_review_1_stem = df_stemming["review"][4] # 1
test_review_2_stem = df_stemming["review"][0] # 5
test_review_3_stem = df_stemming["review"][30] # 2

test_reviews_stem = [test_review_1_stem, test_review_2_stem,test_review_3_stem]

In [ ]:
for test in test_reviews_stem:
  inputs = tokenizer(test, return_tensors='pt', padding=True, truncation=True)
  with torch.no_grad():
      outputs = model_stem(**inputs)
      logits = outputs.logits

  # Получение предсказанного класса
  predictions = torch.argmax(logits, dim=-1)
  print(test)
  print(f"Оценка модели: {predictions[0] + 1}| " + "Оценка отзыва: ", df_stemming[df_stemming["review"] == test]["rate"].values[0])

дорог человек ко случа связыва сбербанк люб страхован год менеджер сбербанк уговор накопительн страхован жизн глаз врал эт вид страхован очен выгодн мал говор будет застрахова ещ инвестицион доход налогов вычет получ люб врем может расторгнут договор получ сумм выгод банк эт мо деньг будут крут итог лет внесен взнос так реш узна скольк получ лет внесен взнос договор позвон услыша внес нных мно взнос вернут эт расторжен окончан действ договор шок расторгл договор итог мно внес нных средств вернул вниман эт прост открыт граб сбербанк люд связыва сбербанк страхован берег сво нерв деньг посещен офис глаз менеджер врал получ рубл истечен лет позвон громк связ сказа получ рубл верьт слов менеджер сбербанк внимательн чита договор
Оценка модели: 5| Оценка отзыва:  1
оформ каск пожалел случ дтп менеджер анастас саватеев быстр оформ направ сервис котор наход далек связ отвеча вопрос больш спасиб побольш так сотрудник буд дальш оформля каск ваш компан
Оценка модели: 4| Оценка отзыва:  5
т страхов

### SBert на обработанных данных (лемматизация)

In [ ]:
tokenizer_stem = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
model_lem = AutoModelForSequenceClassification.from_pretrained("ai-forever/sbert_large_nlu_ru", num_labels=5)  # Укажите количество классов

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ai-forever/sbert_large_nlu_ru and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Пример текста отзыва
test_review_1_lem = df_lematization["review"][4] # 1
test_review_2_lem = df_lematization["review"][0] # 5
test_review_3_lem = df_lematization["review"][30] # 2

test_reviews_lem = [test_review_1_lem, test_review_2_lem,test_review_3_lem]

In [ ]:
for test in test_reviews_lem:
  inputs = tokenizer(test, return_tensors='pt', padding=True, truncation=True)
  with torch.no_grad():
      outputs = model_lem(**inputs)
      logits = outputs.logits

  # Получение предсказанного класса
  predictions = torch.argmax(logits, dim=-1)
  print(test)
  print(f"Оценка модели: {predictions[0] + 1}| " + "Оценка отзыва: ", df_lematization[df_lematization["review"] == test]["rate"].values[0])  # Добавляем 1, чтобы получить класс от 1 до 5


дорогие человеки коем случае связывайтесь сбербанком любому страхованию году менеджер сбербанка уговорила накопительной страхование жизни глаза врала это вид страхования очень выгодный мало говорила будете застрахованы ещ инвестиционный доход налоговый вычет получите любое время можете расторгнуть договор получив сумму выгода банка это мои деньги будут крутиться итоге лет внесения взносов таки решила узнать сколько получу лет внесения взносов договору позвонив услышала внес нных мной взносов вернут это расторжении окончании действия договора шоке расторгла договор итоге мной внес нных средств вернули внимание это просто открытый граб сбербанка люди связывайтесь сбербанком страхованию берегите свои нервы деньги посещение офис глаза менеджер врала получу рублей истечении лет позвонила громкой связи сказали получу рублей верьте словам менеджера сбербанка внимательно читайте договор
Оценка модели: 3| Оценка отзыва:  1
оформили каско пожалели случилось дтп менеджер анастасия саватеева быстр

### Сравнение модели RNN и Bert на задачи классификации текстов

**RNN:**

* RNN хорошо справляются с задачами, где важна последовательность данных, но они менее эффективны для обработки длинных текстов.

* Точность RNN может быть ниже по сравнению с современными моделями.

**SBert:**

* SBert демонстрирует высокую точность на задачах классификации текстов благодаря двунаправленному контексту и механизму внимания.

* Модель может эффективно обрабатывать длинные тексты и учитывать контекстные зависимости.

Несмотря на высокие показатели точности и метрики f1: `99.9`, модель RNN при классификации отзыва к одному из класов показывала результаты хуже, чем предобученная модель SBert.

## Задание 3

Примените один из трансформеров, например BERT, к задаче генерации англоязычного и русскоязычного текстов. Сравните результаты с LSTM. Сделайте выводы.

### Генерация текста

Проверим генерацию текста на модели `GPT-2`

Создаем конвейер генерации текста

In [ ]:
text_generation = pipeline("text-generation")

No model was supplied, defaulted to openai-community/gpt2 and revision 607a30d (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
test_text_1 = "I am speaking to"
test_text_2 = "The world is"
test_text_3 = "Yesterday in Financial university "

In [ ]:
generated_text_1 = text_generation(test_text_1, max_length=50, do_sample=False, truncation=True)[0]
generated_text_2 = text_generation(test_text_2, max_length=50, do_sample=False, truncation=True)[0]
generated_text_3 = text_generation(test_text_3, max_length=50, do_sample=False, truncation=True)[0]

In [ ]:
generated_text_1.get("generated_text")

'I am speaking to you, my dear, and I am not afraid of you. I am afraid of you. I am afraid of you. I am afraid of you. I am afraid of you. I am afraid of you. I am afraid'

Перевод: Я разговариваю с тобой, моя дорогая, и я тебя не боюсь. Я боюсь тебя. Я боюсь тебя. Я боюсь тебя. Я боюсь тебя. Я боюсь тебя. Я боюсь

In [ ]:
generated_text_2.get("generated_text")

"The world is a better place if you're a good person.\n\nI'm not saying that you should be a bad person. I'm saying that you should be a good person.\n\nI'm not saying that you should be a bad"

Перевод: Мир становится лучше, если ты хороший человек.

Я не говорю, что ты должен быть плохим человеком. Я говорю, что ты должен быть хорошим человеком.

Я не говорю, что ты должен быть плохим

In [ ]:
generated_text_3.get("generated_text")

'Yesterday in Financial university \xa0I was asked to write a paper on the subject of the "Growth of the Global Economy" and I was told that I was not allowed to do so. I was told that I was not allowed to write a'

Перевод:Вчера в Финансовом университете меня попросили написать реферат на тему "Рост мировой экономики", и мне сказали, что мне не разрешили этого сделать. Мне сказали, что мне не разрешили написать реферат на тему "Рост мировой экономики".

Попробуем сгенерировать текст на другом языке, например, на русском.

Для этого создадим токенизатор и модель.

In [ ]:
model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

In [ ]:
text_generation = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
test_text_1_rus = "Бедные люди"
test_text_2_rus = "Что для тебя красота"
test_text_3_rus = "Я говорил с ним "

In [ ]:
generated_text_1 = text_generation(test_text_1_rus, max_length=50, do_sample=False)[0]
generated_text_2 = text_generation(test_text_2_rus, max_length=50, do_sample=False)[0]
generated_text_3 = text_generation(test_text_3_rus, max_length=50, do_sample=False)[0]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [ ]:
generated_text_1.get("generated_text")

'Бедные людический простивать простивать прости'

In [ ]:
generated_text_2.get("generated_text")

'Что для тебя красота простический простически'

In [ ]:
generated_text_3.get("generated_text")

'Я говорил с ним с на в на в на в на в на в на в н'

**Выводы:**

Модель демонстрирует более высокое качество генерации текста **на английском языке** по сравнению с русским.

`На английском:`

* Генерируемый текст, хоть и не идеален, сохраняет некоторую связность и смысл на коротких отрезках.

* Однако, с увеличением длины текста, наблюдается повторение слов и фрагментов, а также утрата смысла.

`На русском:`

* Генерация текста крайне ограничена: к исходному слову или предложению добавляется лишь одно слово, которое затем повторяется несколько раз.

* В результате, генерируемый текст не несет никакого смысла.

`Решение:`

Проблема может быть решена путем применения различных схем декодирования, направленных на улучшение разнообразия и логичности генерируемого текста.

### Схема декодирования

Расмотрим схему декодирования `«Лучевой поиск»`

In [ ]:
model_name = 'gpt2'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

In [ ]:
# encode context the generation is conditioned on
model_inputs = [tokenizer(test_text_1, return_tensors='pt'),tokenizer(test_text_2, return_tensors='pt'),tokenizer(test_text_3, return_tensors='pt')]

In [ ]:
# set no_repeat_ngram_size to 2
for input in model_inputs:
  beam_output = model.generate(
      **input,
      max_new_tokens=40,
      num_beams=5,
      no_repeat_ngram_size=2,
      early_stopping=True,
      pad_token_id=tokenizer.eos_token_id
  )

  print("\nOutput:\n" + 100 * '-')
  print(tokenizer.decode(beam_output[0], skip_special_tokens=True))


Output:
----------------------------------------------------------------------------------------------------
I am speaking to you because I want you to know that I am here to tell you that you are not alone.

You are the only one who can help me. I will not let you down. You

Output:
----------------------------------------------------------------------------------------------------
The world is going to be a better place in the next few years," he said.

"I think it's a good time to start thinking about what we can do to make it better."

Output:
----------------------------------------------------------------------------------------------------
Yesterday in Financial university  I had the opportunity to talk to a lot of people who have been in the business for a long time, and they all said that they are very happy with the way things are going. They


Перевод:

**Выход:**

Я обращаюсь к вам, потому что хочу, чтобы вы знали: я здесь, чтобы сказать вам, что вы не одиноки.

Вы единственный, кто может мне помочь. Я вас не подведу. Вы

**Выход:**

В ближайшие несколько лет мир станет лучше", - сказал он.

"Я думаю, сейчас самое время подумать о том, что мы можем сделать, чтобы сделать его лучше".

**Выход:**

Вчера в Финансовом университете у меня была возможность пообщаться со многими людьми, которые давно работают в этом бизнесе, и все они сказали, что очень довольны тем, как идут дела. Они

Рассмотрим схему декодирования `«Top-K Sampling»`

In [ ]:
for input in model_inputs:
  # set top_k to 50
  sample_output = model.generate(
      **input,
      max_new_tokens=40,
      do_sample=True,
      top_k=50,
      pad_token_id=tokenizer.eos_token_id
  )

  print("\nOutput:\n" + 120 * '-')
  print(tokenizer.decode(sample_output[0], skip_special_tokens=True))


Output:
------------------------------------------------------------------------------------------------------------------------
I am speaking to you in the streets of London with the intention of informing you that one of us has been murdered by an individual in an attempt to kill himself. In case you are unaware of where this individual is,

Output:
------------------------------------------------------------------------------------------------------------------------
The world is moving in a direction where we're becoming the largest trading empire in the world. And it's not just China or South Korea. And in the United States, China is now more powerful than ever before

Output:
------------------------------------------------------------------------------------------------------------------------
Yesterday in Financial university  I have a very good knowledge of the situation at HSBC so it would seem that most of the bank's staff were there for a job as a way to increase staff mor

Перевод:

**Выход:**

Я обращаюсь к вам на улицах Лондона с намерением сообщить, что один из нас был убит человеком, пытавшимся покончить с собой. На случай, если вы не знаете, где находится этот человек,


**Выход:**


Мир движется в направлении, в котором мы становимся крупнейшей торговой империей в мире. И это касается не только Китая или Южной Кореи. А в Соединенных Штатах Китай сейчас могущественнее, чем когда-либо прежде

**Выход:**

Вчера в Финансовом университете я очень хорошо изучил ситуацию в HSBC, так что, похоже, большинство сотрудников банка пришли туда на работу, чтобы поднять моральный дух персонала.  На следующий день

In [ ]:
model_name = 'sberbank-ai/rugpt3medium_based_on_gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_gpt3 = GPT2LMHeadModel.from_pretrained(model_name)

In [ ]:
text_generation = pipeline("text-generation", model=model_gpt3, tokenizer=tokenizer)

In [ ]:
model_inputs_rus = [tokenizer(test_text_1_rus, return_tensors='pt'),tokenizer(test_text_2_rus, return_tensors='pt'),tokenizer(test_text_3_rus, return_tensors='pt')]

In [ ]:
generated_text_1 = text_generation(test_text_1_rus, max_length=50, do_sample=False)[0]
generated_text_2 = text_generation(test_text_2_rus, max_length=50, do_sample=False)[0]
generated_text_3 = text_generation(test_text_3_rus, max_length=50, do_sample=False)[0]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
generated_text_1.get("generated_text")

'Бедные люди, бедные люди,\n\t\tКак они несчастны!\n\n\t\tКак они несчастны!\n\t\tКак они несчастны!\n\n\t\tКак они несчастны!\n\t\tКак они несчастны!'

In [ ]:
generated_text_2.get("generated_text")

'Что для тебя красота?\n\n—\xa0Красота — это то, что я вижу.\n\n—\xa0А что для тебя красота?\n\n—\xa0Красота — это то, что я чувствую.\n\n—\xa0А что для'

In [ ]:
generated_text_3.get("generated_text")

'Я говорил с ним \nо том, что он не может быть моим мужем,\nи он сказал, что я не могу быть его женой.\n\nЯ сказал, что я не могу быть его женой,\nи он сказал, что я'

In [ ]:
# set no_repeat_ngram_size to 2
for input in model_inputs_rus:
  beam_output = model_gpt3.generate(
      **input,
      max_new_tokens=40,
      num_beams=5,
      no_repeat_ngram_size=2,
      early_stopping=True,
      pad_token_id=tokenizer.eos_token_id
  )

  print("\nOutput:\n" + 100 * '-')
  print(tokenizer.decode(beam_output[0], skip_special_tokens=True))


Output:
----------------------------------------------------------------------------------------------------
Бедные люди, — сказал он.

— Да, они несчастны. Они не знают, что такое счастье, и не понимают, как оно может на них воздействовать. Им не дано понять

Output:
----------------------------------------------------------------------------------------------------
Что для тебя красота?

— Красота — это то, что я вижу вокруг себя, — ответила она. Она посмотрела на него, и он увидел в ее глазах то же самое выражение, которое он

Output:
----------------------------------------------------------------------------------------------------
Я говорил с ним 
о том, что он не прав.

Я сказал ему:
&quot;Я знаю,
что ты прав, но я не хочу, чтобы ты это знал. Я не могу


**Вывод:**

* Лучевой поиск `(Beam Search)` и `Top-K Sampling` продемонстрировали схожие результаты.

На английском языке:

* Генерируемый текст остается понятным и сохраняет смысл.

На русском языке:

* Более эффективной оказалась модель GPT-3, специально обученная на русском тексте.

* Модель gpt2 не дообучена для генерации текста именно на русском языке. Она способна генерировать что-то похожее на слова, но крайне редко выдаёт слова, которые подходили бы по смыслу.


Для достижения лучших результатов генерации текста на русском языке рекомендуется использовать модели, адаптированные под русскую грамматику и лексику.

### Генерация слов в предложении

In [ ]:
unmasker = pipeline('fill-mask', model='bert-base-cased')

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
predicted = unmasker("I went to [MASK] to buy some candies")
print(list(map(lambda word: word['token_str'], predicted)))

['town', 'work', 'school', 'store', 'Mexico']


In [ ]:
predicted = unmasker("I studyied in [MASK] for 4 year, but I`m still studying")
print(list(map(lambda word: word['token_str'], predicted)))

['college', 'Germany', 'France', 'England', 'English']


In [ ]:
predicted = unmasker("I ate a bowl of [MASK] with a fragrant fresh bread")
print(list(map(lambda word: word['token_str'], predicted)))

['cereal', 'soup', 'rice', 'stew', 'bread']


Для вставки слова в контекст предложения модель bert показывает отличный результат из-за двунаправленной архитектуры. То есть модель учитывает контекст, как до пропущенного слова, так и после. Поэтому варианты генерации такие точные.

In [ ]:
unmasker = pipeline('fill-mask', model='DeepPavlov/rubert-base-cased')

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
predicted = unmasker("Я бы с удовольствием покатался на [MASK] по городу, если бы не снег и пробки" )
print(list(map(lambda word: word['token_str'], predicted)))

['лыжах', 'велосипеде', 'коньках', 'машине', 'трассе']


In [ ]:
predicted = unmasker("Летом мы планируем поехать в [MASK], чтобы отдохнуть на море." )
print(list(map(lambda word: word['token_str'], predicted)))

['Италию', 'Испанию', 'Европу', 'Австралию', 'Японию']


In [ ]:
predicted = unmasker("Моя любимая книга — это [MASK], она вдохновляет меня каждый день." )
print(list(map(lambda word: word['token_str'], predicted)))

['книга', 'любовь', 'поэзия', 'биография', 'фантастика']


Основываясь на предоставленных примерах, модель `DeepPavlov/rubert-base-cased` демонстрирует следующие характеристики:

* Модель успешно интерпретирует контекст предложений и предлагает релевантные замены для [MASK]. Например, в предложении "Я бы с удовольствием покатался на [MASK] по городу, если бы не снег и пробки", модель предлагает варианты "лыжах", "велосипедe", "коньках", "машине", "трассе", что логично с учетом контекста


* Модель демонстрирует гибкость в обработке разных тем и стилей текста

* Предсказания модели релевантны и соответствуют контексту. Однако, в некоторых случаях могут встречаться менее очевидные или менее распространенные варианты

* Несмотря на высокое качество предсказаний, модель может предлагать варианты, которые не всегда идеально подходят к контексту


### Cравнение модели GPT2, Bert и LSTM на задачи генерации текста

**LSTM:**

LSTM обучалась на последовательностях, составленных из слов. В результате генерируемый текст обычно имел смысл, хотя иногда слова могли быть не связаны между собой.

**GPT-2:**

Генерируемый GPT-2 текст, хотя и не идеален, сохраняет связность и смысл на коротких отрезках. Однако с увеличением длины текста наблюдается повторение слов и фрагментов, а также утрата смысла. Это можно улучшить с помощью схем декодирования, что позволяет сохранить текст понятным и содержательным.

**BERT:**

1.   Новый пункт
2.   Новый пункт



BERT демонстрирует отличные результаты в задачах вставки пропущенного слова в контекст предложения благодаря своей двунаправленной архитектуре. Она учитывает контекст как до, так и после пропущенного слова, что делает её эффективной для таких задач.

## Задание 4

Примените один из трансформеров, например BERT, к задаче машинного перевода.

In [ ]:
!pip install sacremoses

In [ ]:
# функция загрузки модели для перевода текста
from transformers import AutoModelForSeq2SeqLM

In [ ]:
model_name = 'Helsinki-NLP/opus-mt-ru-en'
tokenizer_rus_to_eng = AutoTokenizer.from_pretrained(model_name)
model_rus_to_eng = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
model_name = 'Helsinki-NLP/opus-mt-en-ru'
tokenizer_eng_to_rus = AutoTokenizer.from_pretrained(model_name)
model_eng_to_rus = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
model_name = 'Helsinki-NLP/opus-mt-zh-en'
tokenizer_zh_to_eng = AutoTokenizer.from_pretrained(model_name)
model_zh_to_eng = AutoModelForSeq2SeqLM.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/307M [00:00<?, ?B/s]

In [ ]:
model_name = 'Helsinki-NLP/opus-mt-en-zh'
tokenizer_eng_to_zh = AutoTokenizer.from_pretrained(model_name)
model_eng_to_zh = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
def translator(text_to_translate):
  # русский - англ - китайский - англ - русский
  first_text = text_to_translate

  # перевод с русского на английский
  input_ids = tokenizer_rus_to_eng(text_to_translate, return_tensors="pt")
  generated_tokens = model_rus_to_eng.generate(**input_ids)
  result = tokenizer_rus_to_eng.batch_decode(generated_tokens, skip_special_tokens=True)
  text_to_translate = result[0]

  # Перевод с английского на китайский
  input_ids = tokenizer_eng_to_zh(text_to_translate, return_tensors="pt")
  generated_tokens = model_eng_to_zh.generate(input_ids['input_ids'], decoder_start_token_id=model_eng_to_zh.config.pad_token_id)
  result = tokenizer_eng_to_zh.batch_decode(generated_tokens, skip_special_tokens=True)
  text_to_translate = result[0]

  # Перевод с китайского на английский
  input_ids = tokenizer_zh_to_eng(text_to_translate, return_tensors="pt")
  generated_tokens = model_zh_to_eng.generate(input_ids['input_ids'], decoder_start_token_id=model_zh_to_eng.config.pad_token_id)
  result = tokenizer_zh_to_eng.batch_decode(generated_tokens, skip_special_tokens=True)
  text_to_translate = result[0]

  # перевод с английского на русский
  input_ids = tokenizer_eng_to_rus(text_to_translate, return_tensors="pt")
  generated_tokens = model_eng_to_rus.generate(input_ids['input_ids'])
  result = tokenizer_eng_to_rus.batch_decode(generated_tokens, skip_special_tokens=True)
  print(f"Original text: {first_text}\nTranslated text: {result[0]}")
  print()

  return result[0]

In [ ]:
text1 = "Съешь ещё этих мягких французских булок."
text2 = "Счастье — это не конечная цель, а путь, который мы выбираем."
text3 = "Чтение книг развивает воображение и креативность."

In [ ]:
translate1 = translator(text1)
translate2 = translator(text2)
translate3 = translator(text3)

Original text: Съешь ещё этих мягких французских булок.
Translated text: Ешь больше мягкого французского хлеба.

Original text: Счастье — это не конечная цель, а путь, который мы выбираем.
Translated text: Счастье - это не конечная цель, а путь, который мы выбрали.

Original text: Чтение книг развивает воображение и креативность.
Translated text: Чтение книг поощряет воображение и творчество.

